## Week 3 Assignmentment Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install bs4
!pip install folium
!pip install -U numpy
!pip install -U pandas
!pip install -U scipy==1.4.1
!pip install -U scikit-learn
!pip install -U imbalanced-learn
#!pip install plotly

     |████████████████████████████████| 122kB 21.6MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Requirement already up-to-date: numpy in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (1.19.5)
Requirement already up-to-date: pandas in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (1.1.5)
     |████████████████████████████████| 26.1MB 16.4MB/s eta 0:00:01
  Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled scipy-1.5.3
     |████████████████████████████████| 22.3MB 16.4MB/s eta 0:00:01
     |████████████████████████████████| 307kB 59.8MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1
     |████████████████████████████████| 215kB 21.7MB/s eta 0:00:01


In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
from bs4 import BeautifulSoup
print('done')

done


## 1. Scraping Wikipedia page and creating dataframe

In [10]:
# Get the URL and save in a variable named html

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(URL).text

In [11]:
# Parse the html into a BS object
soup = BeautifulSoup(html, 'html5lib')

In [12]:
# Finding which table in the page contains we're looking for

tables = soup.find_all('table')
print(len(tables)) # How many tables are in the page?
tables[0] # Checking if the first (zeroeth) table is the correct one - YES
table = tables[0]

3


In [13]:
# Creating and populating dictionary with info in wikipedia table

table_contents=[]

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [14]:
print(df.shape) # How many postal codes are there?

(103, 3)


## 2. Adding latitude and longitude to the dataframe using Geocoder Python package

In [15]:
!pip install geocoder
import geocoder # import geocoder

In [16]:
postal_codes = df['PostalCode'].tolist()

#print(postal_codes)

latitudes = []
longitudes = []

#for i, postal_code in enumerate(postal_codes):
for postal_code in postal_codes:

    #print('hello')
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        latitudes.append(latitude)
        longitudes.append(longitude)

# Define Lat/Long columns in the existing df and latitude/longitude lists as the source

df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [44]:
df.dtypes

PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [24]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills North,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


## 3. Segmenting and Clustering neighborhoods in Toronto based on postal code as per [instructions](https://www.coursera.org/learn/applied-data-science-capstone/peer/I1bDq/segmenting-and-clustering-neighborhoods-in-toronto) as that is what we have lat/long coordinates for (so no need to split into individual neighborhoods)

#### I'm going to cluster all Toronto postal codes, not just those with 'Toronto' in the borough name. Let's see how they are distributed on the Toronto map.

In [40]:
print('Toronto has {} boroughs across {} postal codes.'.format(len(df['Borough'].unique()),len(df['PostalCode'].unique())))

Toronto has 15 boroughs across 103 postal codes.


In [25]:
# create map of Toronto using latitude and longitude values

import folium
map_toronto = folium.Map(location=[lat, lng], zoom_start=10)

In [26]:
# Loads of commented-out code from my attempts to do the map correctly
#df['Latitude']=df['Latitude'].astype(float)
#df['Longitude']=df['Longitude'].astype(float)

#latitude = float(latitude) for latitude in df['Latitude']
#longitude = float(longitude) for longitude in df['Longitude']

# add markers to map including postal code, Borough name and neighborhood
for postcode, lat, lng, borough, neighborhood in zip(df['PostalCode'], df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{},{},{}'.format([postcode], [borough ], neighborhood)
    label = folium.Popup(label.replace("'","") , parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [27]:
# Define Foursquare credentials and version

CLIENT_ID = 'Q0N0FAEK1FAOWW4INGJAD2UHVYU31YU3OLL310RPPLIKD1U3' # your Foursquare ID
CLIENT_SECRET = 'OW15WJW2FFHCWEXCGH4TI1I2NGC3NPHSWVSNX10LZLSJUJEK' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q0N0FAEK1FAOWW4INGJAD2UHVYU31YU3OLL310RPPLIKD1U3
CLIENT_SECRET:OW15WJW2FFHCWEXCGH4TI1I2NGC3NPHSWVSNX10LZLSJUJEK


In [28]:
def getNearbyVenues(postal_code, borough, latitude, longitude, radius=500, LIMIT = 100):
    
    venues_list=[]
    for p_code, boro, lat, lng in zip(postal_code, borough, latitude, longitude):
        print(p_code) #p_code isn't passed in GET but aids comprehension of the loop
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            p_code,
            borough, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                'Borough', 
                'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
# Pass relevant columns from Toronto dataframe 'df' into function getNearbyVenues to get output specified by that function

toronto_venues = getNearbyVenues(postal_code=df['PostalCode'], borough = df['Borough'], latitude=df['Latitude'], longitude=df['Longitude']) 

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(2379, 8)


,PostalCode,Borough,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,M3A,0 North York 1 ...,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,0 North York 1 ...,43.75245,-79.32991,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,0 North York 1 ...,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,0 North York 1 ...,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,M4A,0 North York 1 ...,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [31]:
toronto_venues.groupby('PostalCode').count()

,Borough,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,,
M1B,1,1,1,1,1,1,1
M1C,1,1,1,1,1,1,1
M1E,4,4,4,4,4,4,4
M1G,4,4,4,4,4,4,4
M1H,3,3,3,3,3,3,3
M1J,5,5,5,5,5,5,5
M1K,5,5,5,5,5,5,5
M1L,9,9,9,9,9,9,9
M1M,9,9,9,9,9,9,9


In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


In [33]:
# one hot encoding - convert 'Venue Category' column to multiple with dummy variables (1/0)
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
toronto_onehot.shape

(2379, 268)

In [35]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [36]:
toronto_grouped.shape

(102, 268)

In [37]:
num_top_venues = 10

for p_code in toronto_grouped['PostalCode']:
    temp = toronto_grouped[toronto_grouped['PostalCode'] == p_code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues)

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
pcodes_venues_sorted = pd.DataFrame(columns=columns)
pcodes_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    pcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

pcodes_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Film Studio,Field,Farmers Market,Farm,Falafel Restaurant
1,M1C,Bar,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Flower Shop
2,M1E,Tea Room,Construction & Landscaping,Gym / Fitness Center,Park,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
3,M1G,Coffee Shop,Park,Business Service,Korean BBQ Restaurant,Yoga Studio,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
4,M1H,Gaming Cafe,Lounge,Trail,Yoga Studio,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant


### Cluster postcodes (5 clusters using k-means)

In [40]:
from sklearn.cluster import KMeans

# set number of clusters

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 3, 0, 3, 3, 0, 0, 0, 0], dtype=int32)

In [41]:
# add clustering labels

pcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [42]:
toronto_merged = df

# merge toronto_grouped with df to add latitude/longitude for each postal code
toronto_merged = toronto_merged.join(pcodes_venues_sorted.set_index('PostalCode'), on='PostalCode', how = 'inner')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,4,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Yoga Studio
1,M4A,North York,Victoria Village,43.73057,-79.31306,3,Park,German Restaurant,Grocery Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Spa,Event Space,Food Truck,Electronics Store,Restaurant,Bakery,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,0,Clothing Store,Cosmetics Shop,Men's Store,Furniture / Home Store,Restaurant,Women's Store,Toy / Game Store,Bookstore,Smoothie Shop,Leather Goods Store
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188,0,Coffee Shop,Falafel Restaurant,Fried Chicken Joint,Gastropub,Theater,Italian Restaurant,Sandwich Place,Café,Mediterranean Restaurant,Burrito Place


In [43]:
toronto_merged.groupby('Cluster Labels').count()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
1,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [ ]:
# to find out why I was getting 'list indices must be integers or slices, not float' error at next step
#toronto_merged.isnull().sum()

In [44]:
# Visualizing clusters on map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Thrift / Vintage Store,Spa,Event Space,Food Truck,Electronics Store,Restaurant,Bakery,Pub
3,North York,0,Clothing Store,Cosmetics Shop,Men's Store,Furniture / Home Store,Restaurant,Women's Store,Toy / Game Store,Bookstore,Smoothie Shop,Leather Goods Store
4,Queen's Park,0,Coffee Shop,Falafel Restaurant,Fried Chicken Joint,Gastropub,Theater,Italian Restaurant,Sandwich Place,Café,Mediterranean Restaurant,Burrito Place
5,Etobicoke,0,Pharmacy,Bank,Park,Grocery Store,Café,Skating Rink,Shopping Mall,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store
7,North York,0,Gas Station,Coffee Shop,Burger Joint,Park,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
8,East York,0,Pizza Place,Breakfast Spot,Athletics & Sports,Pharmacy,Rock Climbing Spot,Café,Fast Food Restaurant,Flea Market,Bank,Gastropub
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop,Hotel,Middle Eastern Restaurant,Italian Restaurant,Bar,Ramen Restaurant
10,North York,0,Pizza Place,Grocery Store,Bakery,Italian Restaurant,Fast Food Restaurant,Latin American Restaurant,Gas Station,Bank,Japanese Restaurant,Mediterranean Restaurant
11,Etobicoke,0,Pizza Place,Tea Room,Chinese Restaurant,Sandwich Place,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
12,Scarborough,0,Bar,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Flower Shop


## Cluster 2

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,North York,1,Residential Building (Apartment / Condo),Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
45,North York,1,Park,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Yoga Studio,Dry Cleaner
68,Central Toronto,1,Park,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Yoga Studio,Dry Cleaner
69,West Toronto,1,Sandwich Place,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
73,Central Toronto,1,Park,Playground,Gym Pool,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
91,Downtown Toronto,1,Park,Playground,Bike Trail,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
98,Etobicoke,1,Park,Lounge,Farm,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio


## Cluster 3

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,Home Service,Yoga Studio,Farm,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
71,Scarborough,2,Home Service,Auto Garage,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm


## Cluster 4

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3,Park,German Restaurant,Grocery Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
16,York,3,Hockey Arena,Dog Run,Playground,Field,Grocery Store,Park,Trail,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store
17,Etobicoke,3,Park,Fish & Chips Shop,Electronics Store,College Rec Center,Grocery Store,Shopping Mall,Ethiopian Restaurant,Eastern European Restaurant,Elementary School,Escape Room
18,Scarborough,3,Tea Room,Construction & Landscaping,Gym / Fitness Center,Park,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
26,Scarborough,3,Gaming Cafe,Lounge,Trail,Yoga Studio,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
32,Scarborough,3,Indian Restaurant,Spa,Restaurant,Grocery Store,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
35,East York/East Toronto,3,Intersection,Film Studio,Park,Yoga Studio,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
39,North York,3,Trail,Construction & Landscaping,Dog Run,Park,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
49,North York,3,Bakery,Basketball Court,Park,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Yoga Studio
83,Central Toronto,3,Gym,Trail,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant


## Cluster 5

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Yoga Studio
6,Scarborough,4,Fast Food Restaurant,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Film Studio,Field,Farmers Market,Farm,Falafel Restaurant
